<a href="https://colab.research.google.com/github/Michael-David-Lam/Medical-Dialogue-Summary/blob/dev2tariq/Medical_Dialogue_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U evaluate
!pip install -U rouge_score
!pip install -U peft
!pip install sentencepiece
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
g

In [ ]:
import kagglehub
import pandas as pd
import re
!git clone https://github.com/abachaa/MTS-Dialog.git
# Load data
training_data =pd.read_csv('/content/MTS-Dialog/Main-Dataset/MTS-Dialog-TrainingSet.csv')
validation_data = pd.read_csv('/content/MTS-Dialog/Main-Dataset/MTS-Dialog-ValidationSet.csv')
assignment_data = pd.read_csv('/content/MTS-Dialog/Main-Dataset/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv')
# Rename columns
training_data = training_data.rename(columns={'context': 'input_text', 'target': 'target_text'})

from datasets import Dataset
train_dataset = Dataset.from_pandas(training_data)
val_dataset = Dataset.from_pandas(validation_data)
assignment_dataset = Dataset.from_pandas(assignment_data)

Cloning into 'MTS-Dialog'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 98 (delta 18), reused 3 (delta 3), pack-reused 72 (from 1)
Receiving objects: 100% (98/98), 1.19 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_name)
def preprocess_function(examples):
    inputs = examples["dialogue"]
    targets = examples["section_text"]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        targets,
        max_length=512,
        truncation=True,
        padding="max_length"
    ).input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Tokenize datasets
train_tokenized = train_dataset.map(preprocess_function, batched=True) # Changed from training_data to train_dataset
val_tokenized = val_dataset.map(preprocess_function, batched=True) # Changed from validation_data to val_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import Dataset, DataLoader

class DoctorPatientDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_target_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        self.sections = [
            'chief_complaint',
            'history_illness',
            'past_medical_history',
            'diagnosis',
            'treatment',
        ]
    def _create_structured_target(self, row):
        target_parts = []
        for section in self.sections:
            section_key = section.strip("<>")
            if section_key in row and pd.notna(row[section_key]):
                target_parts.append(f"{section} {row[section_key]}")
        return " ".join(target_parts)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        input_text = data_point['input_text']
        target_text = data_point['target_text']

        # Tokenize inputs
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_input_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize targets
        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Replace padding token id with -100 so it's ignored in loss calculation
        target_ids = target_encoding['input_ids']
        target_ids[target_ids == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_ids.flatten(),
            'section_header': data_point['section_header'],
            'id': data_point['id']
        }


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import LoraConfig, get_peft_model, TaskType
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
from peft import LoraConfig, get_peft_model, TaskType
# Example data preparation

# Initialize model with LoRA
model = T5ForConditionalGeneration.from_pretrained(model_name)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, lora_config)  # Wrap base model with LoRA



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def create_dataloaders(processed_data, tokenizer, batch_size=8, train_ratio=0.8):
    # Split data into train and validation sets
    train_size = int(len(processed_data) * train_ratio)
    train_data = processed_data[:train_size]
    val_data = processed_data[train_size:]

    # Create datasets
    train_dataset = DoctorPatientDataset(training_data, tokenizer)
    val_dataset = DoctorPatientDataset(validation_data, tokenizer)

    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [ ]:
assignment_inputs = assignment_data["section_text"].tolist()
assignment_encodings = tokenizer(
    assignment_inputs,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments,  Seq2SeqTrainer , Seq2SeqTrainingArguments
from evaluate import load as load_metric
from peft import LoraConfig, get_peft_model
import numpy as np
import evaluate
import torch
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
     report_to="none",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-ea7945556a48>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-3435b262f1ae>", line 1, in <cell line: 0>
    trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2556, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 3718, in training_step
    loss = self.compute_loss(model, inputs, num_items_in_batch=num_items_in_batch)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/di

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Tokenize assignment data
assignment_inputs = assignment_data["dialogue"].tolist()
assignment_encodings = tokenizer(
    assignment_inputs,
    max_length=512,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
)

# Generate predictions
model.eval()
predictions = model.generate(
    input_ids=assignment_encodings["input_ids"].to(model.device),
    attention_mask=assignment_encodings["attention_mask"].to(model.device),
    max_length=512
)

# Decode predictions
generated_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)

# Display the generated texts
for i, text in enumerate(generated_texts[:5]):  # Show the first 5 generated texts
    print(f"Generated Text {i+1}:\n{text}\n")

# Combine original context with generated texts
submission_df = pd.DataFrame({
    "context": assignment_inputs,  # Original context
    "prediction": generated_texts  # Generated texts
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

print("submission.csv generated successfully!")



Generated Text 1:
, sir. Yes, I just turned fifty five. Doctor: Good afternoon, sir. Yes, I just turned fifty five. Doctor: Well, the M R I demonstrated right contrast temporal mass. Patient: Yes, I do. Well, the M R I demonstrated right contrast temporal mass. Patient: Yes, I do. Well, given this mass, and your new seizures, I am concerned that this could be a high grade glioma, we'll need to do more tests.

Generated Text 2:
Doctor: Do you have medical issues? Patient: Oh yes, stroke. Doctor: Anything else? Patient: Sleep apnea.

Generated Text 3:
, no pain. Doctor: No, no pain. Doctor: How about joint pain? Patient: No, no pain. Doctor: How about joint pain? Patient: Um no, I don't feel any joint pain. Doctor: Okay, good. Doctor: Do you feel any stiffness or weakness in your muscle? Patient: Um, nothing like that. Doctor: Do you have any back pain? Patient: No. Doctor: Okay, good. Doctor: Do you have any back pain? Patient: No. Doctor: Okay, good.

Generated Text 4:
Doctor: Anyone i

In [ ]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams = 5):
  inputs = tokenizer.encode(
      "summarize: " + text,
      return_tensors="pt",
      max_length=max_length,
      truncation=True
  )
  summary_ids = model.generate(
      inputs,
      max_length=150,
      min_length=40,
      length_penalty=2.0,
      num_beams=num_beams,
      early_stopping=True
  )

In [ ]:
rouge = evaluate.load("rouge")

NameError: name 'evaluate' is not defined